In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.dirname(os.getcwd()))

import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

from src.helper import create_and_prepare_dataset, get_data_size

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using GPU")
else:
    print("TensorFlow is using CPU")

TensorFlow is using CPU


In [3]:
PROCESSED_DATA_DIR = "../data/processed"
TRAINING_DATA_DIR = "../data/split/train"
VALIDATION_DATA_DIR = "../data/split/val"
TEST_DATA_DIR = "../data/split/test"

BATCH_SIZE = 5

print(get_data_size(TRAINING_DATA_DIR))
print(get_data_size(VALIDATION_DATA_DIR))

MinMaxScaler()
1660
515


In [4]:
training_dataset = create_and_prepare_dataset(TRAINING_DATA_DIR, BATCH_SIZE)
validation_dataset = create_and_prepare_dataset(VALIDATION_DATA_DIR, BATCH_SIZE)
test_dataset = create_and_prepare_dataset(TEST_DATA_DIR, BATCH_SIZE)

In [5]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(63,)),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(7, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0011),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [6]:
def train_model(model):
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3,
                                                        restore_best_weights=True, min_delta= 0.01)

        model.fit(training_dataset.repeat(),
                steps_per_epoch=int(5* (get_data_size(TRAINING_DATA_DIR) / BATCH_SIZE)),
                validation_data=validation_dataset.repeat(),
                validation_steps=int(5* (get_data_size(VALIDATION_DATA_DIR) / BATCH_SIZE)),
                epochs=100,
                callbacks=[early_stopping]
        ) 

        return model

In [7]:
model = train_model(build_model())

loss, accuracy = model.evaluate(validation_dataset)
print(f"Test accuracy: {accuracy}")
print(f"Test loss: {loss}")

Epoch 1/100
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.4241 - loss: 1.4248 - val_accuracy: 0.6602 - val_loss: 0.9739
Epoch 2/100
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.6304 - loss: 0.9870 - val_accuracy: 0.7301 - val_loss: 0.8367
Epoch 3/100
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.6924 - loss: 0.8188 - val_accuracy: 0.7398 - val_loss: 0.7106
Epoch 4/100
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7199 - loss: 0.7403 - val_accuracy: 0.7456 - val_loss: 0.7508
Epoch 5/100
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7289 - loss: 0.6938 - val_accuracy: 0.7534 - val_loss: 0.6229
Epoch 6/100
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7503 - loss: 0.6445 - val_accuracy: 0.8039 - val_loss: 0.5714
Epoch 7/100
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.7692 - loss: 0.5893 - val_accuracy: 0.7515 - val_loss: 0.7325
Epoch 8/100
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.7551 - loss

C:\Users\imani\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [8]:
choice = input("Do you want to save the model? (y/n): ")

if choice.lower() == 'y':
    model.save(f"../models/model_acc_{accuracy:.2f}_loss_{loss:.2f}.h5")
    print("Model saved successfully!")

Model saved successfully!
